## Dealing with NaN's
### NaN in python

In [1]:
n1 = float("nan")
n2 = float("Nan")
n3 = float("NaN")
n4 = float("NAN")
print(n1, n2, n3, n4)

nan nan nan nan


In [2]:
import math

n1 = math.nan
print(n1)

nan


### NaN in pandas
### Example without NaN

In [36]:
import pandas as pd

df = pd.read_csv('data/temperatures.csv', sep=';', decimal=',')
df[:5]

,time,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
0,06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
1,06:15:00,14.5,14.5,14.0,15.0,14.5,14.7
2,06:30:00,14.6,15.1,14.8,15.3,14.0,14.2
3,06:45:00,14.8,14.5,15.6,15.2,14.7,14.6
4,07:00:00,15.0,14.9,15.7,15.6,14.0,15.3


I want to calculate the average temperatures per measuring point over all the sensors.

In [37]:
df.mean(axis='rows')

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14396\923746142.py:1: FutureWarning: The default value of numeric_only in DataFrame.mean is deprecated. In a future version, it will default to False. In addition, specifying 'numeric_only=None' is deprecated. Select only valid columns or specify the value of numeric_only to silence this warning.
  df.mean(axis='rows')


sensor1    19.775926
sensor2    19.757407
sensor3    19.840741
sensor4    20.187037
sensor5    19.181481
sensor6    19.437037
dtype: float64

In [38]:
average_temp_series = df.mean(axis='columns')
average_temp_series[:5]

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_14396\2729108434.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  average_temp_series = df.mean(axis='columns')


0    13.933333
1    14.533333
2    14.666667
3    14.900000
4    15.083333
dtype: float64

In [43]:
sensors = df.columns.values[1:]
#all columns except time will be removed

df = df.drop(sensors, axis=1)

I'll assign the average temperature values as a new column 'temperature':

In [46]:
df = df.assign(temperature=average_temp_series)
df[:3]

,time,temperature
0,06:00:00,13.933333
1,06:15:00,14.533333
2,06:30:00,14.666667


### Example with NaN

In [49]:
import pandas as pd

temp_df = pd.read_csv("data/temperatures.csv",sep=";", index_col=0, decimal=",")
temp_df[:3]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
06:15:00,14.5,14.5,14.0,15.0,14.5,14.7
06:30:00,14.6,15.1,14.8,15.3,14.0,14.2


Demonstration of how where works

In [53]:
s = pd.Series(range(4))
s.where(s>1)

0    NaN
1    NaN
2    2.0
3    3.0
dtype: float64

In [54]:
import numpy as np
import pandas as pd

A = np.random.randint(1, 30, (4,2))
df = pd.DataFrame(A, columns=['Foo', 'Bar'])
df

,Foo,Bar
0,20,14
1,19,12
2,12,11
3,21,29


In [60]:
-df

,Foo,Bar
0,-20,-14
1,19,-12
2,-12,11
3,21,29


In [59]:
m = df % 2 == 0
df.where(m, -df, inplace=True)
df

,Foo,Bar
0,20,14
1,-19,12
2,12,-11
3,-21,-29


In [67]:
random_df = pd.DataFrame(np.random.random(size=temp_df.shape), columns=temp_df.columns.values, index=temp_df.index)
random_df[:3]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,0.096018,0.177860,0.127828,0.347214,0.242017,0.485551
06:15:00,0.620883,0.119218,0.486433,0.216270,0.901529,0.585665
06:30:00,0.785559,0.091942,0.197307,0.041134,0.388347,0.679264


In [69]:
nan_df = pd.DataFrame(np.nan,columns=temp_df.columns.values, index=temp_df.index)
nan_df[:3]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,NaN,NaN,NaN,NaN,NaN,NaN
06:15:00,NaN,NaN,NaN,NaN,NaN,NaN
06:30:00,NaN,NaN,NaN,NaN,NaN,NaN


In [71]:
df_bool = random_df < 0.8
df_bool[:3]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,True,True,True,True,True,True
06:15:00,True,True,True,True,False,True
06:30:00,True,True,True,True,True,True


In [83]:
disturbed_data = temp_df.where(df_bool, nan_df)
disturbed_data[:3]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
06:15:00,14.5,14.5,14.0,15.0,NaN,14.7
06:30:00,14.6,15.1,14.8,15.3,14.0,14.2


In [74]:
disturbed_data.to_csv('data/temperatures_with_NAN.csv')
disturbed_data[:5]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
06:15:00,14.5,14.5,14.0,15.0,NaN,14.7
06:30:00,14.6,15.1,14.8,15.3,14.0,14.2
06:45:00,14.8,14.5,15.6,15.2,14.7,14.6
07:00:00,15.0,14.9,15.7,15.6,14.0,NaN


### Using Dropna on the DataFrame

In [79]:
df = disturbed_data.dropna()  #if axis=1, dropna drops columns
df[:5]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
06:30:00,14.6,15.1,14.8,15.3,14.0,14.2
06:45:00,14.8,14.5,15.6,15.2,14.7,14.6
07:30:00,15.4,15.3,15.6,15.6,14.7,15.1
08:00:00,15.7,15.6,15.9,16.2,15.4,15.4


I'll take out all the rows, where more than one NaN value occurred.

In [84]:
cleansed_df = disturbed_data.dropna(thresh=5, axis=0)
cleansed_df[:5]

,sensor1,sensor2,sensor3,sensor4,sensor5,sensor6
time,,,,,,
06:00:00,14.3,13.7,14.2,14.3,13.5,13.6
06:15:00,14.5,14.5,14.0,15.0,NaN,14.7
06:30:00,14.6,15.1,14.8,15.3,14.0,14.2
06:45:00,14.8,14.5,15.6,15.2,14.7,14.6
07:00:00,15.0,14.9,15.7,15.6,14.0,NaN


In [92]:
average_temp_series = cleansed_df.mean(axis=1)
sensors = cleansed_df.columns.values
df = cleansed_df.drop(sensors, axis=1)
df = df.assign(temperature=average_temp_series)
df[:5]

,temperature
time,
06:00:00,13.933333
06:15:00,14.540000
06:30:00,14.666667
06:45:00,14.900000
07:00:00,15.040000


## Binning in Python and Pandas
The following function can be used to create bins

In [1]:
def create_bins(lower_bound, width, quantity):
    """
    Create bins returns an equal_width (distance) partitioning.
    It returns an ascending list of tuple, representing the i intervals.
    A tuple bins[i], i.e. (bins[i][0], bins[i][1]) with i > 0 and i < quantity, satisfies the following conditions:
              (1) bins[i][0] + width == bins[i][1]
              (2) bins[i-1][0] + width == bins[i][0] and
              (3) bins[i-1][1] + width == bins[i][1]
    """
    bins = []
    for low in range(lower_bound, lower_bound + quantity*width, width):
        bins.append((low, low + width))
    return bins

In [2]:
bins = create_bins(10, 10, 5)
bins

[(10, 20), (20, 30), (30, 40), (40, 50), (50, 60)]

In [3]:
def find_bin(value, bins):
    """
    bins is a list of tuples, like [(0,20), (20, 40), (40, 60)],
    binning returns the smallest index i of bins so that
    bin[i][0] <= value < bin[i][1]
    
    """
    for i in range(0, len(bins)):
        if bins[i][0] <= value < bins[i][1]:
            return i

In [4]:
from collections import Counter

bins = create_bins(50, 4, 11)
weights_of_persons = [73.4, 69.3, 64.9, 75.6, 74.9, 80.3, 78.6, 84.1, 88.9, 
                      90.3, 83.4, 69.3,52.4, 58.3, 67.4, 74.0, 89.3, 63.4]

binned_weights = []
for value in weights_of_persons:
    bin_index = find_bin(value, bins)
    print(value, bin_index, bins[bin_index])
    binned_weights.append(bin_index)
    
frequencies = Counter(binned_weights)
print(frequencies)

73.4 5 (70, 74)
69.3 4 (66, 70)
64.9 3 (62, 66)
75.6 6 (74, 78)
74.9 6 (74, 78)
80.3 7 (78, 82)
78.6 7 (78, 82)
84.1 8 (82, 86)
88.9 9 (86, 90)
90.3 10 (90, 94)
83.4 8 (82, 86)
69.3 4 (66, 70)
52.4 0 (50, 54)
58.3 2 (58, 62)
67.4 4 (66, 70)
74.0 6 (74, 78)
89.3 9 (86, 90)
63.4 3 (62, 66)
Counter({4: 3, 6: 3, 3: 2, 7: 2, 8: 2, 9: 2, 5: 1, 10: 1, 0: 1, 2: 1})


### Binning with Pandas

In [5]:
import pandas as pd

bins2 = pd.IntervalIndex.from_tuples(bins, closed='left')
bins2

IntervalIndex([[50, 54), [54, 58), [58, 62), [62, 66), [66, 70) ... [74, 78), [78, 82), [82, 86), [86, 90), [90, 94)], dtype='interval[int64, left]')

In [6]:
categorical_object = pd.cut(weights_of_persons, bins2)
print(categorical_object)

[[70, 74), [66, 70), [62, 66), [74, 78), [74, 78), ..., [58, 62), [66, 70), [74, 78), [86, 90), [62, 66)]
Length: 18
Categories (11, interval[int64, left]): [[50, 54) < [54, 58) < [58, 62) < [62, 66) ... [78, 82) < [82, 86) < [86, 90) < [90, 94)]


### Other ways to define bins

In [7]:
categorical_object = pd.cut(weights_of_persons, 18)
print(categorical_object)

[(71.35, 73.456], (69.244, 71.35], (62.928, 65.033], (75.561, 77.667], (73.456, 75.561], ..., (56.611, 58.717], (67.139, 69.244], (73.456, 75.561], (88.194, 90.3], (62.928, 65.033]]
Length: 18
Categories (18, interval[float64, right]): [(52.362, 54.506] < (54.506, 56.611] < (56.611, 58.717] < (58.717, 60.822] ... (81.878, 83.983] < (83.983, 86.089] < (86.089, 88.194] < (88.194, 90.3]]


In [156]:
sequence_of_scalars = [ x[0] for x in bins]
sequence_of_scalars.append(bins[-1][1])